<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson08_nearestNeighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#we will train a k-nearest neighbors classifier to identify malware.

In [1]:
#No tf/keras needed yet
#yay! plots! I need to learn these ...

from sklearn import neighbors
import matplotlib.pyplot as plt

In [2]:
#Data come from Great Courses
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/malware-test.csv
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/data/malware-train.csv

--2023-01-05 10:19:56--  https://github.com/mlittmancs/great_courses_ml/raw/master/data/malware-test.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/malware-test.csv [following]
--2023-01-05 10:19:57--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/data/malware-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11593257 (11M) [text/plain]
Saving to: ‘malware-test.csv’

malware-test.csv    100%[===================>]  11.06M  --.-KB/s    in 0.04s   

2023-01-05 10:19:57 (258 MB/s) - ‘malware-test.csv’ saved [11593257/11593257

In [12]:
#declare a getdat function, which collects the data and labels within the file used for training. 
#We loop over the lines of the given file one by one, splitting each line into components at the commas. 
#The data file uses pe-malicious to label the positive instances of malware. 
#So, we use == "pre-malicious" to turn those labels into 0/False for safe and 1/True for malware. 
#The function returns the data and the labels for the file. Since the data is read from the file as a string, 
#each component needs to be converted to a floating point number.
#We then use this funciton to get the training and test data used for our model.

#This cell is only a function definition

def getdat(filename):

    #So with closes resources after using them and is often used when opening a file.
    #I guess it eliminates the f after data is created?
    #no f is a TextIOWrapper and seems to still exist after
    #It is supposed to be like a try/catch block and relate to exception handling somehow too
    with open(filename, "r") as f:
       data = f.readlines()

    #anyway, now we have data, which is a list with 4000 items, each of which is a string of comma separated data

    #Initial lists we will use for output (brackets define a list)
    dat = []
    labs = []

    #Loops through each line and takes the string and splits it into data 
    for line in data:

        #Splits each line into a new list along the commas. Wordline should have 487 items
        wordline = line.split(",")

        #append to label list (+ can append to a list apparently) "True" (not a one) if the first item in this line is "pe-malicious"
        #does it authomatically pad a False otherwise, even for elements past the last true?
        labs = labs + [wordline[0] == "pe-malicious"]

        #mini-loop through all the data items in wordline, converting each to float and then appending as new items to data list
        dat = dat + [[float(wordline[i]) for i in range(1,len(wordline))]]

    #Return lists of labels and datalines (float)
    return(dat,labs)


In [11]:
#Skip this cell. It just tests what elements in the function definition do / look like

with open("malware-train.csv", "r") as f:
       data = f.readlines()

      #It's a list from one of these GC files so contains a string with comma separated data values and its hard to say how many without split (for me anyway)
       test = data[0]

      
       test01 = test.split(",")

       print(test01[0])

labs = []


pe-legit


In [15]:
traindat, trainlabs = getdat("malware-train.csv")
testdat, testlabs = getdat("malware-test.csv")

#traindat should be a 4000 item list
print(traindat[0])
#trainlabs should be a 4000 item list
print(trainlabs[0])
#testdat should be a 2000 item list
print(testdat[0])
#testlabs should be a 2000 item list
print(testlabs[0])

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.266666666667, 0.141176470588, 0.0627450980392, 0.313725490196, 0.341176470588, 1.0, 0.0823529411765, 0.439215686275, 0.443137254902, 0.250980392157, 0.0, 0.41568627451, 0.0823529411765, 0.333333333333, 0.333333333333, 1.0, 0.454901960784, 0.141176470588, 0.125490196078, 1.0, 0.454901960784, 0.141176470588, 0.125490196078, 0.333333333333, 1.0, 0.207843137255, 0.345098039216, 0.211764705882, 0.258823529412, 0.0, 1.0, 0.0823529411765, 0.533333333333, 0.443137254902, 0.250980392157, 0.0, 0.333333333333, 1.0, 0.462745098039, 0.0470588235294, 0.909803921569, 0.56862745098, 0.839215686275, 1.0, 1.0, 0.223529411765, 0.176470588235, 0.298039215686, 0.211764705882, 0.258823529412, 0.0, 0.458823529412, 0.360784313725, 0.41568627451, 0.0313725490196, 1.0, 0.207843137255, 0.345098039216, 0.211764705882, 0.258823529412, 0.0, 1.0, 0.0823529411765, 0.376470588235, 0.0149752461244, 0.00418444903211, 0.00396123727204, 0.00395763708237, 0.00402809

In [16]:
#define a testscore function, which calculates the number of correctly classified examples in the test set.

#Just the function definition
def testscore(dat,labs):

    #So presumably labs is the same labs as the output of getdat
    #But dat will be a model object clf?

    #genberate labvel preditions
    yhats = clf.predict(dat)

    #Match predictions and labels in usual way
    correct = sum([yhats[i] == labs[i] for i in range(len(dat))])

    #Retrun proportion correct
    return(correct)

In [17]:
#Run nearest neighbor on training data and get accuracy on test data

#Initialise a list to hold accuracy values for each level of k (the number of nearest neighbors considered)
acc = []

#Appears to be number of observations in training to use. 4000 happens to be all of them
m = 4000

for k in [1,5,7,9]:

    #Create configured object for nearest neighbor
    clf = neighbors.KNeighborsClassifier(n_neighbors=k,metric="cosine")

    #fit object (which I guess doesn't do much, as there is no real training)
    clf = clf.fit(traindat[:m], trainlabs[:m])

    #accumulate a list of output for each k, including the k value, the accuracy for training set and accuracy for test set
    acc = acc + [[k, testscore(traindat[:m],trainlabs[:m])/m, testscore(testdat,testlabs)/len(testlabs)]]

In [ ]:
#Good, finally a plot

#plt is from matplotlib.pyplot
#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

#so it's x then y then formats
#I can't just hand acc over directly?

# Plot dependence on k
plt.plot([v[0] for v in acc], [v[1] for v in acc], c="blue")
plt.plot([v[0] for v in acc], [v[2] for v in acc], c="red")
plt.title('malware')
plt.xlabel('k')
plt.ylabel('acc')
#plt.show()

In [19]:
v[0] for v in acc

SyntaxError: ignored